# 🚀 Koushole RAG Processor

**Process NCTB textbooks for RAG using FREE cloud GPU**

**Robust Version:** Handles library version conflicts automatically.

---

**SETUP:**
1. Go to Runtime → Change runtime type → Select **T4 GPU**
2. Click **Runtime → Restart session** (if you ran this before)
3. Run all cells in order
4. Enter API keys when prompted

In [ ]:
#@title 1️⃣ Install Dependencies
!pip install -q --upgrade surya-ocr pdf2image Pillow supabase voyageai transformers torch
!apt-get install -q poppler-utils
print("✅ Dependencies installed!")

In [ ]:
#@title 2️⃣ Enter Credentials
from getpass import getpass

print("🔑 Enter your credentials (hidden):")
SUPABASE_URL = input("Supabase URL: ")
SUPABASE_KEY = getpass("Supabase Service Key: ")
VOYAGE_API_KEY = getpass("Voyage AI API Key: ")

if not SUPABASE_URL or not SUPABASE_KEY or not VOYAGE_API_KEY:
    raise ValueError("❌ All credentials are required!")
print("✅ Credentials saved.")

In [ ]:
#@title 3️⃣ Initialize Clients & Models (With Patch)
from supabase import create_client
import voyageai
import sys
from surya.detection import DetectionPredictor
from surya.recognition import RecognitionPredictor

# 1. Setup Clients
try:
    supabase = create_client(SUPABASE_URL, SUPABASE_KEY)
    voyage = voyageai.Client(api_key=VOYAGE_API_KEY)
    print("✅ Clients initialized")
except Exception as e:
    print(f"❌ Client error: {e}")
    sys.exit(1)

# 2. Setup Surya OCR
print("\n🔮 detailed setup for Surya OCR...")
try:
    # Load Detection Model
    print("  -> Loading DetectionPredictor...")
    det_predictor = DetectionPredictor()
    
    # 🩹 MONKEY PATCH: Fix for Transformers version mismatch
    # The 'EfficientViTConfig' in newer transformers might miss 'bbox_size'
    # needed by Surya. We inject it manually to prevent the crash.
    config = det_predictor.model.config
    if not hasattr(config, "bbox_size"):
        print("  ⚠️ Patching missing 'bbox_size' in model config...")
        # Use image_size from config or default to 1024 (standard for Surya)
        input_size = getattr(config, "image_size", [1024, 1024])
        # If input_size is a list/tuple, take the first dim
        config.bbox_size = input_size[0] if isinstance(input_size, (list, tuple)) else input_size
        print("     ✅ Patch applied.")
    
    # Load Recognition Model (passing the detector)
    print("  -> Loading RecognitionPredictor...")
    rec_predictor = RecognitionPredictor(det_predictor)
    print("✅ Surya OCR Models Loaded Successfully")

except Exception as e:
    print(f"❌ Model Loading Error: {e}")
    sys.exit(1)

In [ ]:
#@title 4️⃣ Helper Functions
from pdf2image import convert_from_bytes
import requests
from tqdm import tqdm
from PIL import Image

def extract_text_surya(pdf_bytes):
    """Extract text from PDF using Surya OCR"""
    try:
        images = convert_from_bytes(pdf_bytes, dpi=150)
        all_text = []
        batch_size = 5
        
        pbar = tqdm(total=len(images), desc="OCR Processing")
        for i in range(0, len(images), batch_size):
            batch = images[i:i+batch_size]
            
            # Run detection
            det_results = det_predictor(batch)
            
            # Run recognition (passing detection results improves accuracy)
            # Note: Surya v0.17+ accepts (images, langs, detection_results)
            rec_results = rec_predictor(batch, ["bn", "en"], det_results)
            
            for page in rec_results:
                txt = "\n".join([l.text for l in page.text_lines])
                all_text.append(txt)
            
            pbar.update(len(batch))
        pbar.close()
        return "\n\n".join(all_text)
        
    except Exception as e:
        print(f"OCR Failed: {e}")
        return ""

def chunk_text(text, chunk_size=2000, overlap=200):
    chunks = []
    start = 0
    while start < len(text):
        end = start + chunk_size
        chunk = text[start:end]
        if chunk.strip():
            chunks.append(chunk.strip())
        start = end - overlap
        if start >= len(text) - overlap:
            break
    return [c for c in chunks if len(c) > 50]

def generate_embeddings(chunks):
    if not chunks: return []
    embeddings = []
    batch_size = 20
    for i in tqdm(range(0, len(chunks), batch_size), desc="Generating Embeddings"):
        batch = [c[:8000] for c in chunks[i:i+batch_size]]
        res = voyage.embed(batch, model="voyage-multilingual-2", input_type="document")
        embeddings.extend(res.embeddings)
    return embeddings

print("✅ Helper functions ready!")

In [ ]:
#@title 5️⃣ Fetch Pending Books
print("📡 Fetching pending books from Supabase...")

# Get library books
lib_res = supabase.table('library_books').select('id, title, file_url').or_('chunks_generated.is.null,chunks_generated.eq.false').execute()
lib_books = lib_res.data or []

# Get official books
off_res = supabase.table('official_resources').select('id, title, file_url').or_('chunks_generated.is.null,chunks_generated.eq.false').execute()
off_books = off_res.data or []

all_books = [(b, 'library') for b in lib_books] + [(b, 'official') for b in off_books]

print(f"📚 Library books pending: {len(lib_books)}")
print(f"📖 Official books pending: {len(off_books)}")
print(f"📊 Total to process: {len(all_books)}")

In [ ]:
#@title 6️⃣ Process Loop 🚀
import time

def process_book(book, source_type):
    title = book['title']
    print(f"\n📘 Processing: {title}")
    
    # 1. Download
    try:
        print("  📥 Downloading...")
        r = requests.get(book['file_url'])
        if r.status_code != 200:
            print("  ❌ Download failed")
            return False
        pdf_bytes = r.content
        print(f"  📄 Size: {len(pdf_bytes) / 1024 / 1024:.2f} MB")
    except Exception as e:
        print(f"  ❌ Download error: {e}")
        return False

    # 2. OCR
    text = extract_text_surya(pdf_bytes)
    if not text or len(text) < 100:
        print(f"  ❌ OCR extracted too little text ({len(text)} chars)")
        return False
    print(f"  ✅ Extracted {len(text)} characters")

    # 3. Chunk
    chunks = chunk_text(text)
    print(f"  📦 Generated {len(chunks)} chunks")

    # 4. Embed
    try:
        embeddings = generate_embeddings(chunks)
    except Exception as e:
        print(f"  ❌ Embedding error: {e}")
        return False

    # 5. Store
    print("  💾 saving to database...")
    id_col = 'library_book_id' if source_type == 'library' else 'resource_id'
    
    try:
        # Clear old chunks
        supabase.table('book_chunks').delete().eq(id_col, book['id']).execute()
        
        # Insert NEW chunks
        batch = []
        for idx, (txt, emb) in enumerate(zip(chunks, embeddings)):
            batch.append({
                id_col: book['id'],
                'chunk_index': idx,
                'chunk_text': txt,
                'embedding': emb
            })
            if len(batch) >= 50:
                supabase.table('book_chunks').insert(batch).execute()
                batch = []
        if batch:
            supabase.table('book_chunks').insert(batch).execute()
            
        # Update Status
        table = 'library_books' if source_type == 'library' else 'official_resources'
        supabase.table(table).update({
            'chunks_generated': True,
            'total_chunks': len(chunks),
            'is_processed': True
        }).eq('id', book['id']).execute()
        
        print("  ✅ SUCCESS!")
        return True
        
    except Exception as e:
        print(f"  ❌ Database error: {e}")
        return False

# --- MAIN LOOP ---
success_count = 0
fail_count = 0

for book, source in all_books:
    if process_book(book, source):
        success_count += 1
    else:
        fail_count += 1
    time.sleep(1)

print(f"\n{'='*40}")
print(f"🏁 DONE! Success: {success_count}, Failed: {fail_count}")

## 🎉 All Done!

Check your website chat - the new books should now be searchable!